In [17]:
include("../src/MLTrain.jl")
include("../src/Data_prep.jl")
include("../src/Utils.jl")



layer=Chain(
    DistanceLayer(1,nothing),
    MyLayer(1,1,50.0f0,52.0f0)
)

dist=DistanceLayer(1,nothing)

Data=Float32[3.0717585 -0.14971268 3.0603335; 3.210517 -0.16328853 3.0112257;;; 3.03046 -0.04373725 0.010446698; 3.1080205 0.08581428 -0.24876891;;; 2.877253 -0.025660131 -0.40077204; 3.0260594 -0.054103684 -0.2785623]


layer(Data)


Sono qui


2×1 Matrix{Float32}:
 2.6052947
 1.4102374